### 디렉토리 생성

In [ ]:
import os 
import shutil
num=1
if not os.path.exists('./history'):
    os.makedirs('./history')
while(True):
    PHASE='phase'+str(num)
    day='./history/'+PHASE
    if not os.path.exists(day):
        os.makedirs(day)
        shutil.move('./'+PHASE+'.csv',day+'/'+PHASE+'.csv')
        break
    else:
        num+=1

### WIKI ID 추출

In [ ]:
import mkwikidata
import json
import pandas as pd
import time

In [ ]:
phase1=pd.read_csv('./history/phase1/phase1.csv')
phase2=pd.read_csv('./history/phase2/phase2.csv')
phase3=pd.read_csv('./history/phase3/phase3.csv')
phase4=pd.read_csv('./history/phase4/phase4.csv')
phase5=pd.read_csv('./history/phase5/phase5.csv')
phase6=pd.read_csv('./history/phase6/phase6.csv')
phase7=pd.read_csv('./history/phase7/phase7.csv')
phase8=pd.read_csv('./history/phase8/phase8.csv')
phase9=pd.read_csv('./history/phase9/phase9.csv')

sp480=pd.merge(phase1,phase2)
sp480=pd.merge(sp480,phase3)
sp480=pd.merge(sp480,phase4)
sp480=pd.merge(sp480,phase5)
sp480=pd.merge(sp480,phase6)
sp480=pd.merge(sp480,phase7)
sp480=pd.merge(sp480,phase8)
sp480=pd.merge(sp480,phase9)
sp480

In [ ]:
none_ticker={'BALL':'Q4034815','BF-B':'Q392221','BRK-B':'Q217583','KDP':'Q3116111','WTW':'Q21189883'}

In [ ]:
ticker_symbol=dict()
count=0
for ticker in sp480:
    query='''
    SELECT ?company

    WHERE {
    
    ?company p:P414[pq:P249 '%s'] .
    
    SERVICE wikibase:label{
        bd:serviceParam wikibase:language "en"  .
        }
    }
    '''%ticker
    try:
        query_result=mkwikidata.run_query(query,params={})
        ticker_symbol[ticker]=query_result['results']['bindings'][0]['company']['value'][31:]
    except:
        try:
            query_result=mkwikidata.run_query(query,params={})
            ticker_symbol[ticker]=query_result['results']['bindings'][0]['company']['value'][31:]
        except:
            if ticker not in none_ticker.keys():
                print(query_result)
                print(ticker)
            else:
                ticker_symbol[ticker]=none_ticker[ticker]
    time.sleep(2)

In [ ]:
with open('./history/wiki_ID.json','w') as f:
    json.dump(ticker_symbol,f,indent=2)

### timestamp와 id 추출하기

In [ ]:
import json
with open('./history/wiki_ID.json') as f:
    ticker=json.load(f)

In [ ]:
import requests
relation=['P6379', 'P749', 'P366', 'P739', 'P121', 'P4950', 
'P131', 'P495', 'P625', 'P355', 'P463', 'P5009', 'P17', 'P414',
 'P1830', 'P452', 'P306', 'P176', 'P169', 'P199', 'P1344', 'P156',
  'P2770', 'P3320', 'P113', 'P1454', 'P159', 'P740', 'P114', 'P31',
   'P1056', 'P1889', 'P166', 'P400', 'P361', 'P155', 'P127', 'P112', 'P793']
SYMBOL=dict()
S = requests.Session()

URL = "https://www.wikidata.org/w/api.php"

for Symbol,ID in ticker.items():
    
    PARAMS = {
        "action": "query",
        "format":"json",
        "prop": "revisions",
        "titles": ID,
        "rvprop": "ids|timestamp|flags|comment|user",
        "rvlimit":40000
    }

    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()

    PAGES = DATA["query"]["pages"]
    # PAGES
    day_id=list()
    for page in PAGES.keys():
        for i in range(len(PAGES[page]['revisions'])):
            for r in relation:
                try: 
                    property="[Property:"+str(r)+"]"
                    if property in  PAGES[page]['revisions'][i]['comment']:
                        sample=dict()
                        sample['timestamp']=str(PAGES[page]['revisions'][i]['timestamp'])
                        sample['revid']=PAGES[page]['revisions'][i]['revid']
                        day_id.append(sample)
                except:
                    continue
    SYMBOL[Symbol]=list(reversed(day_id))


### data정리

In [ ]:
import datetime as dt
def add_data(year,month,revid,sample_list):
    sample=dict()
    d=dt.datetime(year,month,1,1)
    sample["timestamp"]=d.strftime('%Y-%m')
    sample["revid"]=revid
    sample_list.append(sample)

In [ ]:
new_json=dict()
for TICKER,DATA in SYMBOL.items():
    sample_list=list()
    year=2014
    month=1
    revid=DATA[0]['revid']
    for data in DATA:
        json_year=int(data['timestamp'][:4])
        json_month=int(data['timestamp'][5:7])
        if year==2014 and json_year==2015:
            while json_month!=month:
                add_data(2015,month,revid,sample_list)
                month+=1
            year=json_year
        elif year!=2014 and  json_year==year and json_month>month:
            while json_month!=month:
                add_data(year,month,revid,sample_list)
                month+=1
            year=json_year
        elif year!=2014 and json_year>year:
            while year!=json_year:
                while month!=12:
                    add_data(year,month,revid,sample_list)
                    month+=1
                add_data(year,month,revid,sample_list)
                year+=1
                month=1
            while month!=json_month:
                add_data(year,month,revid,sample_list)
                month+=1
        revid=data['revid']
    while year!=dt.datetime.today().year:
        while month!=12:
            add_data(year,month,revid,sample_list)
            month+=1
        add_data(year,month,revid,sample_list)
        year+=1
        month=1
    while month!=dt.datetime.today().month:
        add_data(year,month,revid,sample_list)
        month+=1
    new_json[TICKER]=sample_list

In [ ]:
new_json['ZTS']

### 월별로 ID 정리

In [ ]:
import json

TOTAL_DATA=dict()
for i in range(96):
    sample=dict()
    for TICKER,DATA in new_json.items():
        date=DATA[i]['timestamp']
        sample[TICKER]=DATA[i]['revid']
    TOTAL_DATA[date]=sample

In [ ]:
with open('./history/timestamp.json','w',encoding='UTF-8') as file:
    json.dump(TOTAL_DATA,file,indent=1)

### revid로 과거 트리플 추출

In [ ]:
import json
import pandas as pd
with open ('./history/timestamp.json','r') as f:
    DATE=json.load(f)

In [ ]:
"""
    get_pages_revisions.py

    MediaWiki API Demos
    Demo of `Revisions` module: Get revision data with content for pages
    with titles [[API]] and [[Main Page]]

    MIT License
"""
relation=['P6379', 'P749', 'P366', 'P739', 'P121', 'P4950', 
'P131', 'P495', 'P625', 'P355', 'P463', 'P5009', 'P17', 'P414',
 'P1830', 'P452', 'P306', 'P176', 'P169', 'P199', 'P1344', 'P156',
  'P2770', 'P3320', 'P113', 'P1454', 'P159', 'P740', 'P114', 'P31',
   'P1056', 'P1889', 'P166', 'P400', 'P361', 'P155', 'P127', 'P112', 'P793']
import requests
import json
import csv
VERSION					= "0.1"
USER_AGENT			= "Mozilla/5.0 (compatible; Awena/"+VERSION+"; +https://github.com/sedthh/awena-wikidata-crawler/)"
# S = requests.Session()
triple_list=list()
year=2015
day_month=['2018-01','2018-12','2019-06','2019-12','2020-06','2020-12','2021-06','2021-12','2022-06']
for period in day_month:
    for revid in DATE[period].values():
        URL = "https://www.wikidata.org/w/api.php"
        headers	= {
                    "User-Agent"	: USER_AGENT
                }
        PARAMS = {
                "action":'parse',
                "format":"json",
                "oldid":revid,
                "prop":"wikitext"
            }
        R=requests.get("https://www.wikidata.org/w/api.php",headers=headers,params=PARAMS)
        # R = S.get(url=URL, params=PARAMS)

        INFO = R.json()
        DATA=json.loads(INFO['parse']['wikitext']['*'])
        for r in DATA['claims'].keys():
            if r in relation:
                for i in range(len(DATA['claims'][r])):
                    sample=dict()
                    sample['wiki_code']=DATA['id']
                    sample['relation']=r
                    try:
                        sample['node']=DATA['claims'][r][i]['mainsnak']['datavalue']['value']['id']
                        triple_list.append(sample)
                    except:
                        continue
    df=pd.DataFrame(triple_list,columns=['wiki_code','relation','node'])
    df.to_csv(day+'/triple_'+period+'.csv')
    # file_name='./data/triple_'+period+'.csv'
    # triple=open(file_name,'w',encoding='UTF-8')
    # fieldnames=['wiki_code','relation','node']
    # writer=csv.DictWriter(triple,fieldnames=fieldnames)
    # writer.writeheader()
    # writer.writerows(triple_list)
    year+=1
                
    # DATA['parse']['wikitext']['*']
# DATA=DATA['parse']['wikitext']['*']
# data_list=DATA.split()

# PAGES = DATA["query"]["pages"]
# PAGES
# for page in PAGES:
#     print(page[0])

### file 생성

In [ ]:
import json
import csv
import pandas as pd
with open("./"+day+"/wiki_ID.json") as f:
    ticker_entityID=json.load(f) 
data=[]
for key in ticker_entityID.keys():
    sample={}
    sample['ticker']=key
    sample['wiki_code']=ticker_entityID[key]
    data.append(sample)
df=pd.DataFrame(data,columns=['ticker','wiki_code'])
df.to_csv('./'+day+'/company_480.csv',index=False)